In [ ]:
import librosa
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Reshape
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Path to dataset
DATA_PATH = "archive (1)/AudioWAV"  # Update with actual path

# Emotion Mapping (based on filenames)
EMOTION_MAPPING = {
    "ANG": "Anger",
    "DIS": "Disgust",
    "FEA": "Fear",
    "HAP": "Happy",
    "SAD": "Sadness",
    "SUR": "Surprise",
    "NEU": "Neutral"
}
EMOTIONS = list(EMOTION_MAPPING.values())  # Define emotion classes

# Fixed time-step length
MAX_PAD_LENGTH = 100  # Ensures uniform input size

# Feature Extraction Function
def extract_features(file_path, max_pad_length=MAX_PAD_LENGTH):
    y, sr = librosa.load(file_path, sr=22050)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)  # Shape: (40, time_steps)

    # Pad or truncate MFCC to a fixed shape (40, 100)
    if mfccs.shape[1] < max_pad_length:
        pad_width = max_pad_length - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs = mfccs[:, :max_pad_length]

    return np.expand_dims(mfccs, axis=-1)  # Keep it in (40, 100, 1)

# Load dataset
X, y = [], []
for file in os.listdir(DATA_PATH):
    if file.endswith(".wav"):
        file_path = os.path.join(DATA_PATH, file)
        parts = file.split("_")  # Extract emotion from filename
        if len(parts) > 2:
            emotion_label = parts[2]  # Third part of filename (e.g., "SAD")
            if emotion_label in EMOTION_MAPPING:
                features = extract_features(file_path)  # Shape (40, 100, 1)
                X.append(features)
                y.append(EMOTIONS.index(EMOTION_MAPPING[emotion_label]))

# Convert lists to NumPy arrays
X = np.array(X)  # Shape: (num_samples, 40, 100, 1)
y = to_categorical(np.array(y), num_classes=len(EMOTIONS))

# Ensure data is loaded
if len(X) == 0:
    raise ValueError("No audio files were processed. Check dataset path or structure.")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Corrected Model Architecture
model = Sequential([
    Reshape((40, 100, 1), input_shape=(40, 100, 1)),  # Ensure correct shape
    Conv2D(32, (3, 3), activation="relu", padding="same"),  # No TimeDistributed needed
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(len(EMOTIONS), activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train Model
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Save Model
model.save("emotion_model.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.2555 - loss: 11.1935 - val_accuracy: 0.2881 - val_loss: 2.5108
Epoch 2/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.4419 - loss: 1.5278 - val_accuracy: 0.3378 - val_loss: 2.2556
Epoch 3/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5152 - loss: 1.3124 - val_accuracy: 0.3345 - val_loss: 2.0549
Epoch 4/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5753 - loss: 1.1470 - val_accuracy: 0.3539 - val_loss: 1.7621
Epoch 5/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6763 - loss: 0.8978 - val_accuracy: 0.4056 - val_loss: 1.6830
Epoch 6/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6952 - loss: 0.8295 - val_accuracy: 0.3848 - val_loss: 1.8542
Epoch 7/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7685 - loss: 0.6503 - val_accuracy: 0.3109 - val_loss: 3.7635
Epoch 8/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7570 - loss: 0.6582 - val_accuracy: 

In [ ]:
!unzip archive\ \(1\).zip -d archive\ \(1\)

Archive:  archive (1).zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of archive (1).zip or
        archive (1).zip.zip, and cannot find archive (1).zip.ZIP, period.
